In [1]:
#读取MNIST数据

from __future__ import print_function
import tensorflow as tf

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
# 基础cnn分类模型

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [5]:
# 使用gpu/cpu训练模型

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 4s 74us/step - loss: 0.0252 - acc: 0.9929 - val_loss: 0.0277 - val_acc: 0.9913
Epoch 2/12
60000/60000 [==============================] - 4s 68us/step - loss: 0.0236 - acc: 0.9923 - val_loss: 0.0255 - val_acc: 0.9920
Epoch 3/12
60000/60000 [==============================] - 4s 71us/step - loss: 0.0240 - acc: 0.9927 - val_loss: 0.0276 - val_acc: 0.9919
Epoch 4/12
60000/60000 [==============================] - 4s 72us/step - loss: 0.0220 - acc: 0.9930 - val_loss: 0.0262 - val_acc: 0.9915
Epoch 5/12
60000/60000 [==============================] - 4s 68us/step - loss: 0.0207 - acc: 0.9934 - val_loss: 0.0302 - val_acc: 0.9924
Epoch 6/12
60000/60000 [==============================] - 4s 68us/step - loss: 0.0210 - acc: 0.9936 - val_loss: 0.0256 - val_acc: 0.9924
Epoch 7/12
60000/60000 [==============================] - 4s 70us/step - loss: 0.0197 - acc: 0.9937 - val_loss: 0.0281 - val_acc

In [ ]:
# 使用tpu训练模型

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

tpu_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

def rand_batch(batch_size):
    max_offset = x_train.shape[0] - batch_size
    while True:
        offset = np.random.randint(0, max_offset)
        yield x_train[offset:offset+batch_size], y_train[offset:offset + batch_size]

batch_num = (x_train.shape[0]-1)//batch_size + 1
def seq_batch(batch_size):
    def seq_batch_once():
        for i in range(batch_num):
            offset = i * batch_size
            yield x_train[offset:offset+batch_size], y_train[offset:offset + batch_size]
    while True:
        for batch in seq_batch_once():
            yield batch

tpu_model.fit_generator(
    seq_batch(batch_size),
    epochs=epochs,
    steps_per_epoch=batch_num,
    validation_data=(x_test, y_test)    
)